# Transform elevations from the ellipsoid to the EGM96 geoid vertical reference

In [ ]:
import xarray as xr
import pyproj
import os
import glob
import rioxarray as rxr
import sys
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj.crs import CompoundCRS
from tqdm.auto import tqdm
import pandas as pd
import geopandas as gpd
from shapely import wkt
from ast import literal_eval

In [ ]:
# -----Define paths in directory
# path to study-sites
study_sites_path = '/Users/raineyaberle/Google Drive/My Drive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/study-sites/'
# path to snow-cover-mapping
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'
# add path to functions
sys.path.insert(1, base_path+'functions/')
import pipeline_utils as f

In [ ]:
# -----Grab list of site names in study_sites_path
site_names = sorted(os.listdir(study_sites_path))
site_names = [x for x in site_names if not x.startswith('.')]
# only include sites with snowlines
site_names = [x for x in site_names if len(glob.glob(study_sites_path + x + '/imagery/snowlines/*.csv')) > 0]
# only include sites with ArcticDEM geoid files
site_names = [x for x in site_names if os.path.exists(study_sites_path + x + '/DEMs/' + x + '_ArcticDEM_clip_geoid.tif')]
print(str(len(site_names)) + ' study sites:')
site_names

## Transform elevations for snowlines that used ArcticDEM Mosaic or USGS DEMs

In [ ]:
# -----Iterate over sites
bgotus_site_names = ['Wolverine', 'Gulkana', 'LemonCreek', 'SouthCascade', 'Sperry'] # BGOTUS
    
for site_name in site_names[8:9]:

    print(site_name)

    # load AOI
    AOI_path = study_sites_path + site_name + '/AOIs/'
    AOI_fn = glob.glob(AOI_path + '*_outline.shp')[0]
    AOI = gpd.read_file(AOI_fn)
    AOI_WGS = AOI.to_crs('EPSG:4326')
    AOI_WGS_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                        AOI_WGS.geometry[0].centroid.xy[1][0]]
    # grab the optimal UTM zone EPSG code
    epsg_UTM = f.convert_wgs_to_utm(AOI_WGS_centroid[0], AOI_WGS_centroid[1])

    # Check if AOI intersects with ArcticDEM coverage
    intersects = arcticdem_coverage.geometry[0].intersects(AOI_WGS.to_crs('EPSG:'+str(arcticdem_coverage_crs)).geometry[0])
    if (not intersects) and (site_name not in bgotus_site_names):
        print('Outside ArcticDEM coverage and not BGOTUS, continuing... \n\n')
        continue
        
    # Load DEM
    DEM_path = os.path.join(study_sites_path, site_name, 'DEMs')
    DEM_fn = glob.glob(DEM_path + '*.tif')[0]
    

    # load classified image and snowline file names
    snowline_fns = sorted(glob.glob(study_sites_path + site_name + '/imagery/snowlines/*.csv'))
    snowline_fns = [x for x in snowline_fns if '_adj' not in x]

    # iterate over snowlines
    for snowline_fn in tqdm(snowline_fns[0:10]):
        
        # define adjusted file name and check if it exists
        snowline_adj_fn = snowline_fn[0:-3] + '_adj.csv'
        if os.path.exists(snowline_adj_fn):
            continue

        # load snowline
        try:
            snowline = pd.read_csv(snowline_fn)  
        except:
            print('error opening ' + snowline_fn.split('/')[-1] + ', skipping...')
            continue
        
        # skip if vertical reference already exists in attributes
        if 'VerticalReference' in snowline.keys():
            continue
        
        snowline_adj = snowline.to_crs('EPSG:4326') # reproject to WGS84 horizontal coordinates

        
        # Initialize an empty array for the transformed elevation data
        # sl_elevs = snowline_adj['elevation'].data[0]  # Extract the elevation data as a NumPy array

#         # Iterate through each point and transform the coordinates
#         elevation_geoid = np.empty_like(elevation)
#         lon, lat = im_classified['x'].values, im_classified['y'].values
#         lon_mesh, lat_mesh = np.meshgrid(lon, lat)
#         x, y, elevation_geoid = vertical_transformer.transform(lon_mesh, lat_mesh, elevation)
#         elevation_geoid[elevation_geoid < 0] = np.nan

#         im_classified['elevation'] = (('time', 'y', 'x'), [elevation_geoid]) # Replace the elevation data with the transformed data

        # plot results
        # fig, ax = plt.subplots(1, 3, figsize=(24,6))
        # elev1 = ax[0].imshow(elevation, extent=(np.min(im_classified.x.data), np.max(im_classified.x.data), 
        #                                         np.min(im_classified.y.data), np.max(im_classified.y.data)))
        # ax[0].set_title('Ellipsoid height')
        # fig.colorbar(elev1, ax=ax[0], shrink=0.5, label='m.a.s.l.')
        # elev2 = ax[1].imshow(im_classified.elevation.data[0], extent=(np.min(im_classified.x.data), np.max(im_classified.x.data), 
        #                                                               np.min(im_classified.y.data), np.max(im_classified.y.data)))
        # ax[1].set_title('EGM96 height')
        # fig.colorbar(elev2, ax=ax[1], shrink=0.5, label='m.a.s.l.')
        # diff = ax[2].imshow(elevation - im_classified.elevation.data[0], cmap='Reds', 
        #                     extent=(np.min(im_classified.x.data), np.max(im_classified.x.data),
        #                             np.min(im_classified.y.data), np.max(im_classified.y.data)))
        # ax[2].set_title('Difference')
        # fig.colorbar(diff, ax=ax[2], shrink=0.5, label='meters')
        # plt.show()

        # adjust values from float to int for saving
#         im_classified_adj = im_classified.classified.data.astype(float)
#         im_classified_adj[im_classified_adj==0] = np.nan
#         im_classified['classified'] = (('time', 'y', 'x'), im_classified_adj)
#         im_classified = im_classified.fillna(-9999).astype(int)

#         # define elevation source attribute
#         if site_name in bgotus_site_names:
#             elevation_source = "USGS Benchmark Glacier Project: Most recent Digital Elevation Model available for site in the 2022 (Version 7.0) glacier-wide mass balance data release (https://doi.org/10.5066/F7HD7SRF)"
#         else:
#             elevation_source = "ArcticDEM Mosaic (https://developers.google.com/earth-engine/datasets/catalog/UMN_PGC_ArcticDEM_V3_2m_mosaic)"

#         # assign attributes
#         im_classified = im_classified.assign_attrs({'Description': 'Classified image',
#                                                     'Classes':'1 = Snow, 2 = Shadowed snow, 4 = Ice, 5 = Rock, 6 = Water',
#                                                     'HorizontalReference': 'WGS84 (EPSG:4326)',
#                                                     'VerticalReference': 'EGM96 height (EPSG:5773)',
#                                                     'ElevationSource': elevation_source,
#                                                     'NoDataValues': '-9999'
#                                                    })
#         # save to file
#         im_classified.to_netcdf(im_classified_adj_fn, mode='w')
#         # print('adjusted image saved to file: ' + im_classified_adj_fn)

    print(' ')

In [ ]:
DEM_fn = glob.glob(DEM_path + '/*.tif')[0]
DEM = xr.open_dataset(DEM_fn)
elevation = DEM.band_data.data[0]
DEM = DEM.drop_dims('band')
DEM['elevation'] = (('y', 'x'), elevation)
DEM

In [ ]:
plt.imshow(DEM.elevation.data, cmap='terrain', clim=(1000,3000),
           extent=(np.min(DEM.x.data)/1e3, np.max(DEM.x.data)/1e3, np.min(DEM.y.data)/1e3, np.max(DEM.y.data)/1e3))
plt.colorbar()
plt.show()

In [ ]:
snowline = pd.read_csv(snowline_fns[10])
snowline['geometry'] = snowline['geometry'].apply(wkt.loads)
snowline_gdf = gpd.GeoDataFrame(snowline, geometry=snowline['geometry'], crs=snowline['CRS'][0])
snowline_gdf

In [ ]:
# reproject to WGS84 in geometry
snowline_gdf = snowline_gdf.to_crs('EPSG:4326')

# Initialize an empty array for the transformed elevation data
# snowline_elevs = np.zeros(len(snowline['snowline_coords_X']))
    
# sample elevations from DEM at coordinates
snowline_gdf['snowlines_coords_X'][0]

## Transform elevations for classified images that used ArcticDEM Mosaic or USGS DEMs

In [ ]:
# -----Define source and target CRS
src_crs = pyproj.CRS(4979)  # Define the source CRS (WGS 84 ellipsoid)
target_crs = CompoundCRS(name="WGS 84 + EGM96 height", components=["EPSG:4326", "EPSG:5773"]) # Define the target CRS with EGM96 geoid as the vertical datum
vertical_transformer = pyproj.Transformer.from_crs(src_crs, target_crs, always_xy=True) # Create a Pyproj Transformer object to perform the vertical transformation

print('Source CRS:')
print(str(src_crs))
print(' ')
print('Target CRS:')
print(str(target_crs))

In [ ]:
# -----Iterate over sites
bgotus_site_names = ['Wolverine', 'Gulkana', 'LemonCreek', 'SouthCascade', 'Sperry'] # BGOTUS
    
for site_name in site_names[20:]:

    if site_name=='LemonCreek':
        continue
        
    print(site_name)

    # load AOI
    AOI_path = study_sites_path + site_name + '/AOIs/'
    AOI_fn = glob.glob(AOI_path + '*_outline.shp')[0]
    AOI = gpd.read_file(AOI_fn)
    AOI_WGS = AOI.to_crs('EPSG:4326')
    AOI_WGS_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                        AOI_WGS.geometry[0].centroid.xy[1][0]]
    # grab the optimal UTM zone EPSG code
    epsg_UTM = f.convert_wgs_to_utm(AOI_WGS_centroid[0], AOI_WGS_centroid[1])

    # Check if AOI intersects with ArcticDEM coverage
    intersects = arcticdem_coverage.geometry[0].intersects(AOI_WGS.to_crs('EPSG:'+str(arcticdem_coverage_crs)).geometry[0])
    if (not intersects) and (site_name not in bgotus_site_names):
        print('Outside ArcticDEM coverage and not BGOTUS, continuing... \n\n')
        continue

    # load classified image and snowline file names
    im_classified_fns = sorted(glob.glob(study_sites_path + site_name + '/imagery/classified/*.nc'))
    im_classified_fns = [x for x in im_classified_fns if '_adj' not in x]

    # iterate over classified images
    for im_classified_fn in tqdm(im_classified_fns):
        
        # define adjusted file name and check if it exists
        im_classified_adj_fn = im_classified_fn[0:-3] + '_adj.nc'
        if os.path.exists(im_classified_adj_fn):
            continue

        # load classified image
        try:
            im_classified = xr.open_dataset(im_classified_fn)  
        except:
            print('error opening ' + im_classified_fn.split('/')[-1] + ', skipping...')
            continue
        
        # skip if vertical reference already exists in attributes
        if 'VerticalReference' in im_classified.attrs.keys():
            continue
        
        im_classified = im_classified.rio.write_crs('EPSG:'+str(epsg_UTM))
        im_classified = im_classified.rio.reproject('EPSG:4326') # reproject to WGS84 horizontal coordinates

        # Initialize an empty array for the transformed elevation data
        elevation = im_classified['elevation'].data[0]  # Extract the elevation data as a NumPy array

        # Iterate through each point and transform the coordinates
        elevation_geoid = np.empty_like(elevation)
        lon, lat = im_classified['x'].values, im_classified['y'].values
        lon_mesh, lat_mesh = np.meshgrid(lon, lat)
        x, y, elevation_geoid = vertical_transformer.transform(lon_mesh, lat_mesh, elevation)
        elevation_geoid[elevation_geoid < 0] = np.nan

        im_classified['elevation'] = (('time', 'y', 'x'), [elevation_geoid]) # Replace the elevation data with the transformed data

        # plot results
        # fig, ax = plt.subplots(1, 3, figsize=(24,6))
        # elev1 = ax[0].imshow(elevation, extent=(np.min(im_classified.x.data), np.max(im_classified.x.data), 
        #                                         np.min(im_classified.y.data), np.max(im_classified.y.data)))
        # ax[0].set_title('Ellipsoid height')
        # fig.colorbar(elev1, ax=ax[0], shrink=0.5, label='m.a.s.l.')
        # elev2 = ax[1].imshow(im_classified.elevation.data[0], extent=(np.min(im_classified.x.data), np.max(im_classified.x.data), 
        #                                                               np.min(im_classified.y.data), np.max(im_classified.y.data)))
        # ax[1].set_title('EGM96 height')
        # fig.colorbar(elev2, ax=ax[1], shrink=0.5, label='m.a.s.l.')
        # diff = ax[2].imshow(elevation - im_classified.elevation.data[0], cmap='Reds', 
        #                     extent=(np.min(im_classified.x.data), np.max(im_classified.x.data),
        #                             np.min(im_classified.y.data), np.max(im_classified.y.data)))
        # ax[2].set_title('Difference')
        # fig.colorbar(diff, ax=ax[2], shrink=0.5, label='meters')
        # plt.show()

        # adjust values from float to int for saving
        im_classified_adj = im_classified.classified.data.astype(float)
        im_classified_adj[im_classified_adj==0] = np.nan
        im_classified['classified'] = (('time', 'y', 'x'), im_classified_adj)
        im_classified = im_classified.fillna(-9999).astype(int)

        # define elevation source attribute
        if site_name in bgotus_site_names:
            elevation_source = "USGS Benchmark Glacier Project: Most recent Digital Elevation Model available for site in the 2022 (Version 7.0) glacier-wide mass balance data release (https://doi.org/10.5066/F7HD7SRF)"
        else:
            elevation_source = "ArcticDEM Mosaic (https://developers.google.com/earth-engine/datasets/catalog/UMN_PGC_ArcticDEM_V3_2m_mosaic)"

        # assign attributes
        im_classified = im_classified.assign_attrs({'Description': 'Classified image',
                                                    'Classes':'1 = Snow, 2 = Shadowed snow, 4 = Ice, 5 = Rock, 6 = Water',
                                                    'HorizontalReference': 'WGS84 (EPSG:4326)',
                                                    'VerticalReference': 'EGM96 height (EPSG:5773)',
                                                    'ElevationSource': elevation_source,
                                                    'NoDataValues': '-9999'
                                                   })
        # save to file
        im_classified.to_netcdf(im_classified_adj_fn, mode='w')
        # print('adjusted image saved to file: ' + im_classified_adj_fn)

    print(' ')

In [ ]:
site_name = 'Wolverine'

# load classified image and snowline file names
im_classified_fns = sorted(glob.glob(study_sites_path + site_name + '/imagery/classified/*_adj.nc'))

# iterate over classified images
for im_classified_fn in im_classified_fns:

    # load classified image
    im_classified = xr.open_dataset(im_classified_fn)
    im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
    
    fig, ax = plt.subplots(1, 2, figsize=(12,6))
    ax[0].imshow(im_classified['classified'].data[0], clim=(0,5),
                 extent=(np.min(im_classified.x.data), np.max(im_classified.x.data), 
                         np.min(im_classified.y.data), np.max(im_classified.y.data)))
    ax[1].imshow(im_classified['elevation'].data[0], cmap='terrain',
                 extent=(np.min(im_classified.x.data), np.max(im_classified.x.data), 
                         np.min(im_classified.y.data), np.max(im_classified.y.data)))
    plt.show()

## For images that used NASADEM, reproject to WGS84 and add image attributes

In [ ]:
bgotus_site_names = ['Wolverine', 'Gulkana', 'LemonCreek', 'SouthCascade', 'Sperry'] # BGOTUS

# Adjust classified images that use the NASADEM
for i, site_name in enumerate(site_names):
        
    print(site_name)

    # load AOI
    AOI_path = study_sites_path + site_name + '/AOIs/'
    AOI_fn = glob.glob(AOI_path + '*_outline.shp')[0]
    AOI = gpd.read_file(AOI_fn)
    AOI_WGS = AOI.to_crs('EPSG:4326')
    AOI_WGS_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                        AOI_WGS.geometry[0].centroid.xy[1][0]]
    # grab the optimal UTM zone EPSG code
    epsg_UTM = f.convert_wgs_to_utm(AOI_WGS_centroid[0], AOI_WGS_centroid[1])

    # Check if AOI intersects with ArcticDEM coverage
    intersects = arcticdem_coverage.geometry[0].intersects(AOI_WGS.to_crs('EPSG:'+str(arcticdem_coverage_crs)).geometry[0])
    if intersects | (site_name in bgotus_site_names):
        print('Not NASADEM site, continuing... \n')
        continue

    # load classified image and snowline file names
    im_classified_fns = sorted(glob.glob(study_sites_path + site_name + '/imagery/classified/*.nc'))
    im_classified_fns = [x for x in im_classified_fns if '_adj' not in x]

    # iterate over classified images
    for im_classified_fn in tqdm(im_classified_fns):
        
        # define adjusted file name and check if it exists
        im_classified_adj_fn = im_classified_fn[0:-3] + '_adj.nc'
        if os.path.exists(im_classified_adj_fn):
            continue
            
        try:
            im_classified = xr.open_dataset(im_classified_fn) 
        except:
            print('error with ' + im_classified_fn + ', skipping...')
            continue
        im_classified = im_classified.rio.write_crs('EPSG:'+str(epsg_UTM))
        im_classified = im_classified.rio.reproject('EPSG:4326') # reproject to WGS84 horizontal coordinates
        im_classified = xr.where(im_classified==0, -9999, im_classified)
        im_classified = im_classified.rio.write_crs('EPSG:4326')

        # assign attributes
        elevation_source = 'NASADEM (https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001)'
        im_classified = im_classified.assign_attrs({'Description': 'Classified image',
                                                    'Classes':'1 = Snow, 2 = Shadowed snow, 4 = Ice, 5 = Rock, 6 = Water',
                                                    'HorizontalReference': 'WGS84 (EPSG:4326)',
                                                    'VerticalReference': 'EGM96 height (EPSG:5773)',
                                                    'ElevationSource': elevation_source,
                                                    'NoDataValues': '-9999'
                                                   })
        
        # save to file
        im_classified.to_netcdf(im_classified_adj_fn, mode='w')
        

In [ ]:
# sites with errors: 
# ['Emmons', 'Gulkana', 'LemonCreek', 'RGI60-01.08288', 'RGI60-01.08296', 'RGI60-02.04363', 'RGI60-02.06862', 'RGI60-02.12435']

In [ ]:
missing_site_names = []
for site_name in site_names:
        
    im_classified_path = study_sites_path + site_name + '/imagery/classified/'
    im_classified_fns = [x for x in glob.glob(im_classified_path + '*.nc') if '_adj' not in x]
    im_classified_adj_fns = glob.glob(im_classified_path + '*_adj.nc')
    if len(im_classified_fns) != len(im_classified_adj_fns):
        missing_site_names.append(site_name)

print(missing_site_names)

## Delete non-adjusted classified images, rename adjusted 

In [ ]:
for site_name in site_names:
    
    print(site_name)
    
    im_classified_path = study_sites_path + site_name + '/imagery/classified/'
    im_classified_fns = [x for x in glob.glob(im_classified_path + '*.nc') if '_adj' not in x]
    im_classified_adj_fns = glob.glob(im_classified_path + '*_adj.nc')
    
    if (len(im_classified_fns) > 0) and (len(im_classified_adj_fns)>0):
        for im_classified_fn in im_classified_fns:
            os.remove(im_classified_fn)
        
    if len(im_classified_adj_fns) > 0:
        
        for im_classified_adj_fn in im_classified_adj_fns:
            im_classified_adj_fn_new = im_classified_adj_fn.replace('_adj', '')
            os.rename(im_classified_adj_fn, im_classified_adj_fn_new)

## Remove elevation band from classified images

In [ ]:
# -----Iterate over sites
for site_name in site_names:
        
    print('\n' + site_name)

    # load classified image and snowline file names
    im_classified_fns = sorted(glob.glob(study_sites_path + site_name + '/imagery/classified/*.nc'))
    im_classified_fns = [x for x in im_classified_fns if '_adj' not in x]

    # iterate over classified images
    for im_classified_fn in tqdm(im_classified_fns):
        
        # skip if adjusted image already exists in file
        im_classified_adj_fn = im_classified_fn[0:-3] + '_adj.nc'
        if os.path.exists(im_classified_adj_fn):
            continue

        # load classified image
        try:
            im_classified = xr.open_dataset(im_classified_fn)  
        except:
            print('error opening ' + im_classified_fn.split('/')[-1] + ', skipping...')
            continue
            
        # drop elevation band, edit attributes
        im_classified_adj = im_classified.drop('elevation')
        im_classified_adj.attrs = {'Description': 'Classified image',
                               'Classes':'1 = Snow, 2 = Shadowed snow, 4 = Ice, 5 = Rock, 6 = Water',
                               'HorizontalReference': 'WGS84 (EPSG:4326)',
                               'NoDataValues': '-9999'
                              }
        
        # resave classified image
        im_classified_adj.to_netcdf(im_classified_adj_fn, mode='w')
        
